In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer, make_column_selector

from imblearn.pipeline import Pipeline as ImbPipeline

from sklearn.preprocessing import OneHotEncoder, LabelEncoder

from sklearn.model_selection import train_test_split, learning_curve
from sklearn.metrics import classification_report, f1_score, precision_recall_curve, confusion_matrix
from sklearn.metrics import balanced_accuracy_score
from imblearn.metrics import geometric_mean_score
from sklearn.ensemble import RandomForestClassifier
from imblearn.under_sampling import RandomUnderSampler

from sklearn.feature_selection import SelectKBest, VarianceThreshold, chi2

from sklearn.kernel_approximation import RBFSampler
from sklearn.linear_model import SGDClassifier

In [2]:
acc_dtypes = {
    "place":"category",
    "catu":"category",
    "grav":"category",
    "sexe":"category",
    "trajet":"category",
    "locp":"category",
    "actp":"category",
    "etatp":"category",
    "secuUn":"category",
    "secuDeux":"category",
    "tranches_ages":"category",
    "catr":"category",
    "circ":"category",
    "vosp":"category",
    "prof":"category",
    "plan":"category",
    "surf":"category",
    "infra":"category",
    "situ":"category",
    "senc":"category",
    "obs":"category",
    "obsm":"category",
    "choc":"category",
    "manv":"category",
    "catv_Label":"category",
    "lum":"category",
    "agg":"category",
    "int":"category",
    "atm":"category",
    "col":"category",
    "jour_de_la_semaine":"category",
    "heure":"category",
    "dep": "category"
    }
df = pd.read_csv(r"C:\Users\maill\Documents\GitHub\SARA\data\fusion3.csv", low_memory=False)
df['date'] = pd.to_datetime(df['date'])
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df = df.drop(['Unnamed: 0','num_acc','an_nais','an_naiss','age_acc_an','num_veh','senc','occutc','permis','secuDeux','date','com'], axis=1)
df['place'] = df['place'].astype('object')
df['dep'] = df['dep'].replace({'2A':201,'2B':202})
df['dep'] = df['dep'].astype('int64')
                               
df = df.dropna()

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2291745 entries, 0 to 2291796
Data columns (total 34 columns):
 #   Column              Dtype  
---  ------              -----  
 0   place               object 
 1   catu                object 
 2   grav                object 
 3   sexe                object 
 4   trajet              object 
 5   locp                object 
 6   actp                object 
 7   etatp               object 
 8   secuUn              object 
 9   tranches_ages       object 
 10  catr                object 
 11  circ                object 
 12  nbv                 float64
 13  vosp                object 
 14  prof                object 
 15  plan                object 
 16  surf                object 
 17  infra               object 
 18  situ                object 
 19  obs                 object 
 20  obsm                object 
 21  choc                object 
 22  manv                object 
 23  catv_Label          object 
 24  lum                 obje

In [33]:
X = df.drop('grav',axis=1)
y = df['grav']
# X, _, y, _ = train_test_split(X, y, stratify=y, test_size=0.5)  # 50% des données
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [37]:
categorical_variable = make_column_selector(dtype_exclude=np.number)
transformer = make_column_transformer((OneHotEncoder(), categorical_variable))

model = ImbPipeline(steps=[
    ('transformer', transformer),
    ('undersampling', RandomUnderSampler()),
    ('sel_feat', VarianceThreshold(threshold = 0.00001)),
    ('sel_feat2', SelectKBest(score_func=chi2, k=20)),
    ('classifier', RandomForestClassifier(random_state=42,verbose=2))], verbose=2)

# Evaluation

In [38]:
def evaluation(model):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite']))
    print(classification_report(y_test, y_pred))
    print(balanced_accuracy_score(y_test, y_pred))
    print(geometric_mean_score(y_test, y_pred))

In [39]:
evaluation(model)

C:\Users\maill\anaconda3\lib\site-packages\imblearn\utils\_param_validation.py:522: FutureWarning: Passing an int for a boolean parameter is deprecated in version 1.2 and won't be supported anymore in version 1.4.
  warnings.warn(


[Pipeline] ....... (step 1 of 5) Processing transformer, total=  13.9s
[Pipeline] ..... (step 2 of 5) Processing undersampling, total=   6.1s
[Pipeline] .......... (step 3 of 5) Processing sel_feat, total=   0.1s
[Pipeline] ......... (step 4 of 5) Processing sel_feat2, total=   0.6s


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 100


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.7s remaining:    0.0s


building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  1.4min finished


[Pipeline] ........ (step 5 of 5) Processing classifier, total= 1.4min


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    7.5s finished


Classe prédite      Blessé hospitalisé  Blessé léger  Indemne    Tué
Classe réelle                                                       
Blessé hospitalisé               19422         23347    15409  35765
Blessé léger                     22906         62189    52920  26676
Indemne                           3349         18216   142334  23654
Tué                               1689          1339     1310   7824
                    precision    recall  f1-score   support

Blessé hospitalisé       0.41      0.21      0.27     93943
      Blessé léger       0.59      0.38      0.46    164691
           Indemne       0.67      0.76      0.71    187553
               Tué       0.08      0.64      0.15     12162

          accuracy                           0.51    458349
         macro avg       0.44      0.50      0.40    458349
      weighted avg       0.57      0.51      0.52    458349

0.49664199316028834
0.44184549199507445


# Optimisation

# Precision Recall Curve

In [ ]:
precision, recall, threshold = precision_recall_curve(y_test, grid.best_estimator_.decision_function(X_test))

plt.plot(threshold, precision[:-1], label='precision')
plt.plot(threshold, recall[:-1], label='recall')
plt.legend()

In [ ]:
def model_final(model, X, threshold=0):
    return model.decision_function(X) > threshold

In [ ]:
y_pred = model_final(grid.best_estimator_, X_test, threshold=-1)